In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch
from torch import Tensor

from pathlib import Path
from config import Config

config = Config(
    path_to_data=Path('/home/anuiel/Remote/Anuiel/dl-bdz2/data')
)

In [5]:
from dataset import Multi228k
from tokenizer import TextTransform, PAD_IDX

train_dataset = Multi228k(config.path_to_data, 'train', config.source_language, config.target_language)
val_dataset = Multi228k(config.path_to_data, 'val', config.source_language, config.target_language)
test_dataset = Multi228k(config.path_to_data, 'test1', config.source_language, config.target_language)

text_transform = TextTransform(config.path_to_data, config.source_language, config.target_language)
source_vocab_size = len(text_transform.vocab_transform[config.source_language])
target_vocab_size = len(text_transform.vocab_transform[config.target_language])
source_vocab_size, target_vocab_size

(38931, 26346)

In [6]:
from model import load_model

model = load_model(config, source_vocab_size, target_vocab_size, Path('/home/anuiel/Remote/Anuiel/dl-bdz2/data/models/k9gjga8c.pth')).to(config.device)

/home/anuiel/Remote/Anuiel/dl-bdz2/env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [117]:
s = 'diesen morgen werde ich niemals vergessen .'

In [210]:
from tokenizer import BOS_IDX, EOS_IDX
from train import generate_square_subsequent_mask
from inference import greedy_decode
from beam_search import beam_search


def translate(
    model,
    src_sentence: str,
    text_transform: TextTransform,
    config: Config
):
    model.eval()
    src = text_transform.text_transform[config.source_language](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = torch.zeros(num_tokens, num_tokens, dtype=torch.bool)
    # tgt_tokens: Tensor = greedy_decode(
    #     model, src, src_mask, max_len=10, start_symbol=BOS_IDX, device=config.device
    # ).flatten()
    tgt_tokens: Tensor = beam_search(
        model,
        width=1,
        length_penalty=2,
        temp=1,
        src=src,
        src_mask=src_mask,
        max_len=num_tokens+5,
        start_symbol=BOS_IDX,
        device=config.device
    )
    return tgt_tokens
    # return " ".join(
    #     text_transform.vocab_transform[config.target_language].lookup_tokens(list(tgt_tokens.cpu().numpy()))
    # ).replace("<bos>", "").replace("<eos>", "").replace("<unk>", "")

In [211]:
for score, t in translate(model, s, text_transform, config):
    ss = " ".join(
        text_transform.vocab_transform[config.target_language].lookup_tokens(list(t.cpu().numpy()))
    )
    print(ss)

<bos> that morning , i 'll never forget this morning . <eos>


In [21]:
torch.tensor(1).unsqueeze(0)

tensor([1])